In [ ]:
# STEP 1: Install Dependencies
!pip install -q transformers accelerate sentence-transformers faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 109.2 MB/s eta 0:00:00


In [ ]:
# STEP 2: Imports and Model Setup
import pandas as pd
import faiss
import torch
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load LLM model and tokenizer
model_id = "deepseek-ai/deepseek-llm-7b-chat"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/594 [00:00<?, ?B/s]

pytorch_model.bin.index.json: 0.00B [00:00, ?B/s]

...

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.85G [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.97G [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [ ]:
# ✅ STEP 3: Load Dataset (uploaded manually in Colab)
df = pd.read_csv("/content/Indian_Food_Dataset_With_Recipes.csv")
df = df.dropna(subset=["Cleaned-Ingredients", "TranslatedRecipeName", "FullRecipe"])

# Combine fields for better retrieval
df["retrieval_text"] = (
    df["TranslatedRecipeName"] + " | " +
    df["Cleaned-Ingredients"] + " | " +
    df["FullRecipe"]
)

In [ ]:
# ✅ STEP 4: Embed Recipes for Retrieval
embedder = SentenceTransformer("all-MiniLM-L6-v2")
recipe_embeddings = embedder.encode(df["retrieval_text"].tolist(), show_progress_bar=True)

# Create FAISS index for fast similarity search
index = faiss.IndexFlatL2(recipe_embeddings.shape[1])
index.add(recipe_embeddings)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/186 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [ ]:
# ✅ STEP 5: Recipe Retrieval + LLM Answering
def get_recipe_answer(user_query):
    # Embed user query
    query_embedding = embedder.encode([user_query])
    _, indices = index.search(query_embedding, k=3)

    # Prepare context from top 3 recipes
    context = ""
    for idx in indices[0]:
        row = df.iloc[idx]
        context += f"🍽️ Recipe: {row['TranslatedRecipeName']}\n"
        context += f"⏱️ Cooking Time: {row['TotalTimeInMins']} mins\n"
        context += f"🧂 Ingredients: {row['TranslatedIngredients']}\n"
        context += f"👨‍🍳 Instructions: {row['TranslatedInstructions']}\n\n"

    # Prompt LLM with strict constraints (RAG format)
    prompt = f"""<|system|>You are a recipe assistant. Only use the given context. Do not add or hallucinate any recipes or ingredients.
<|user|>Find recipes based on: {user_query}
Only use the recipes provided below. Display exactly 3 recipes with their name, cooking time, ingredients, and instructions.

Context:
{context}
<|assistant|>"""

    # Tokenize and run inference
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=4096).to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=512, do_sample=False)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
# ✅ STEP 6: Run in Colab – Ask for User Input
user_input = input("Enter ingredients or dish name: ")
response = get_recipe_answer(user_input)
print(response)

Enter ingredients or dish name: spinach, eggs, salt, turmeric powder, red chilli powder


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


<|system|>You are a recipe assistant. Only use the given context. Do not add or hallucinate any recipes or ingredients.
<|user|>Find recipes based on: spinach, eggs, salt, turmeric powder, red chilli powder
Only use the recipes provided below. Display exactly 3 recipes with their name, cooking time, ingredients, and instructions.

Context:
🍽️ Recipe:  No Onion No Garlic Cabbage Kofta Curry
⏱️ Cooking Time: 65 mins
🧂 Ingredients:  1 green chili - chopped, 1/2 tsp turmeric powder, 1 sprig of green coriander - cut, 1/2 tsp garam masala powder,2 cups cabbage - grated, 1/2 cup gram flour, 1/2 tsp red chilli powder, 1/2 tsp cumin powder, turmeric powder - A pinch, 1 to 2 bay leaves, 3 to 4 tbsp cream, 1/2 tsp cardamom - break it, make 3 tomatoes - puree, 1/2 tsp salt - as per taste, oil - as per use, 1/2 tsp hot Masala powder, 1-1 / 2 tbsp coriander powder, 1 tbsp ginger - paste, 1 tbsp red chilli powder, salt - according to taste, 2 tbsp coriander - chopped, 1 cup oil - as per use, 2 tbsp y